# Project Finding Lane Lines

### Describing the pipeline

My pipeline consisted of 7 steps. First, I converted the images to grayscale, then I applied gaussian blur using kernel size 7 to reduce undesirable lines, then I applied canny edge detector and after a ROI was applied but I modified the vertices. Now I can finally use hough transform to find lines in the image.

In order to draw a single line on the left and right lanes, I modified the draw_lines() function for separate points calculating slopes and intercepts, I used slopes positives for right and negatives for left. So I calculate average for slopes and intercepts to fit stable lines. Finally I draw the lines in the initial image.


### Shortcomings

One potential shortcoming would be what would happen when sunlight reflect in the road "creating a new light line".

### Improvements

One potential improvement would be in the segmentation algorithm, maybe using color range would be better.

In [1]:
#importing some useful packages
import os
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from IPython.display import HTML
from moviepy.editor import VideoFileClip

%matplotlib inline

In [2]:
def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    #return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [3]:
def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

In [4]:
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

In [5]:
def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

In [6]:
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img, lines

In [7]:
def draw_lines(img, lines, color=[255, 0, 0], thickness=8):
    # separate left line and right line
    left_line_x = []
    left_line_y = []
    left_slopes = []
    left_intercepts = []
    right_line_x = []
    right_line_y = []
    right_slopes = []
    right_intercepts = []    
    for line in lines:
        for x1, y1, x2, y2 in line:
            slope = (y2 - y1) / (x2 - x1)
            if math.fabs(slope) < 0.5:
                continue            
            if slope < 0.0 and slope > -math.inf:
                left_line_x.extend([x1, x2])
                left_line_y.extend([y1, y2])
                left_slopes.append(slope)
                left_intercepts.append(y1 - slope*x1)
            if slope > 0.0 and slope < math.inf:
                right_line_x.extend([x1, x2])
                right_line_y.extend([y1, y2])
                right_slopes.append(slope)
                right_intercepts.append(y1 - slope*x1)
                
    y_max = img.shape[0]
    y_min = int(y_max * 0.6)
    
    if len(left_slopes) > 0:
        left_slope = np.mean(left_slopes)
        left_intercept = np.mean(left_intercepts)
        x_min_left = int((y_min - left_intercept)/left_slope) 
        x_max_left = int((y_max - left_intercept)/left_slope)
        cv2.line(img, (x_min_left, y_min), (x_max_left, y_max), color, thickness)
    
    if len(right_slopes) > 0:
        right_slope = np.mean(right_slopes)
        right_intercept = np.mean(right_intercepts)
        x_min_right = int((y_min - right_intercept)/right_slope) 
        x_max_right = int((y_max - right_intercept)/right_slope)
        cv2.line(img, (x_min_right, y_min), (x_max_right, y_max), color, thickness)
        
    return img

In [8]:
def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)

In [9]:
def process(image):
    image_gray = grayscale(image)
    image_blur = gaussian_blur(image_gray, 7)
    image_canny = canny(image_blur, 50, 150)
    
    #imshape = image_canny.shape
    #vertices = np.array([[(20,imshape[0]),(450, 330), (520, 330), (imshape[1],imshape[0])]], dtype=np.int32)
    
    width = image.shape[1]
    height = image.shape[0]
    vertices = np.array([[(100,height),(width*.48, height*0.6), (width*.55, height*0.6), (width,height)]], dtype=np.int32)
    
    image_roi = region_of_interest(image_canny, vertices)
    #image_lines, lines = hough_lines(image_roi, 2, np.pi/180, 18, 40, 20)
    image_lines, lines = hough_lines(image_roi, 1, np.pi/180, 15, 30, 10)
    image_final = weighted_img(image_lines, image)
    
    return image_final

In [ ]:
# perform on images
path_imgs = "test_images/"
path_imgs_output = "test_images_output/"
img_files = os.listdir(path_imgs)
for file in img_files:
   image = mpimg.imread(path_imgs + file)
   image_final = process(image)
   mpimg.imsave(path_imgs_output + file, image_final)
    
# perform on videos    
path_videos = "test_videos/"
path_videos_output = "test_videos_output/"
video_files = os.listdir(path_videos)
for file in video_files:
    video = VideoFileClip(path_videos + file)
    video_processed = video.fl_image(process)
    %time video_processed.write_videofile(path_videos_output + file, audio=False)